# El algoritmo del perceptrón, y el error en muestra y fuera de muestra

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Introducción 

Esta práctica tiene como fin poner en relieve las ideas básicas de el aprendizaje, utilizando uno de los métodos de aprendizaje más antiguos y fáciles de implementar.

Esta práctica no intenta ser un remplazo del curso, por lo que se asume que los estudiantes conocen el algoritmo de aprendizaje del perceptrón (PLA por sus sigles en inglés), así como las ideas básicas sobre error en muestra y error fuera de muestra. Para esta práctica se puede realizar el problema en cualquier lenguaje de computación que conozca el estudiante. Sin embargo, se recomienda un lenguaje con capacidades de graficación, tal como Matlab, R y Python (con numpy y matplotlib).

Recordemos que el error fuera de muestra E_o es el error sobre todo el conjunto de puntos del espacio X. Una vez definido el criterio del error, el objetivo del aprendizaje es encontrar una hipótesis g en el conjunto de todas las hipótesis posibles que se pueden hacer con el método de aprendizaje.

Lamentablemente, en todos los casos reales se desconoce E_o y lo más que se puede esperar es aproximarlo a partir del error en muestra E_i, el cual se define como el error medio de un conjunto de datos de aprendizaje disponibles.

Para el caso del perceptrón, E_o es la esperanza que un dato se encuentre mal clasificado, y E_i es el porcentaje de datos mal clasificados por el perceptrón. Tambien sabemos, por lo visto en el curso, que si el conjunto de datos que se utiliza en el algoritmo de PLA es linealmente separable, entonces siempre se tiene una hipótesis final g tal que E_i = 0.

Entonces, ¿Si E_i = 0, podríamos decir que el PLA aprende perfectamente? Desgraciadamente esto es falso. ¿Y eso porqué? Pues porque lo que nos interesa es E_o = 0 y no E_i = 0. Esta práctica tiene como fin dejar clara la diferencia entre E_i y E_o.

Para esto, vamos a hacer un poco de trampa, vamos a suponer que nosotros efectivamente conocemos la función con la que se generaron los datos de aprendizaje, la cual va a ser una función del tipo y = sign(k_0 + k_1 * x_1 + k_2 * x_2). Después vamos a generar datos con esta función, y vamos a estimar g(x) = sign(w_0 + w_1 * x_1 + w_2 * x_2) con el PLA. Así nosotros podemos hacer una estimación suficientemente aproximada de que tan grande es en general la diferencia entre E_i y E_o para diferente número de datos en el conjunto de aprendizaje.

# Práctica a realizar

1.Desarrolla una función modelo_aleatorio en el cual:

a. Se soliciten 4 numeros aleatorios entre el 0 y el 1, a los que llamaremos x1, y1, x2, y2.

b. Se obtenga el valor de los pesos de la recta que pasa entre los dos puntos calculados como: k_2 = 1 k_1 = (y2 - y1) / (x2 - x1) k_0 = y_1 - k_1 * x_1 

de forma que:

k_0, k_1, k_2 = modelo_aleatorio()

In [2]:
def modelo_aleatorio():

    x1 = np.random.uniform(0,1)
    y1 = np.random.uniform(0,1)
    x2 = np.random.uniform(0,1)
    y2 = np.random.uniform(0,1)

    k_2 = 1.0 
    k_1 = (y2 - y1) / (x2 - x1) 
    k_0 = y1 - k_1 * x1 

    return k_0, k_1, k_2


print "1: modelo aleatorio: \n"
k_0, k_1, k_2 = modelo_aleatorio()

print k_0
print k_1
print k_2

1: modelo aleatorio: 

0.236179178028
-0.332257291045
1.0


2.Desarrolla una función genera_datostal que reciba un número entero positivo N y devuelva una matriz X de Nrenglones y 2 columnas de manera que los valores de la matriz sean datos aleatorios en el intervalo [0, 1].

In [3]:
def genera_datos(N):

    if N<= 0:
        return None
    else:
        return np.column_stack((np.ones(N), (np.random.uniform(0, 1, size=(N, 2)))))

print "2: genera datos: \n"
X = genera_datos(10)
print X

2: genera datos: 

[[ 1.          0.7355206   0.09880162]
 [ 1.          0.31491358  0.30476456]
 [ 1.          0.2875462   0.7013981 ]
 [ 1.          0.27025677  0.34224153]
 [ 1.          0.67263943  0.34556336]
 [ 1.          0.42539381  0.50327849]
 [ 1.          0.97436584  0.45209885]
 [ 1.          0.12456663  0.68941286]
 [ 1.          0.75656776  0.24895105]
 [ 1.          0.21887156  0.90918474]]


3.Desarrolla una función discriminante_lineal tal que reciba tres valores k_0 k_1 y k_2 y una matriz X de 2 por N y devuelva un vector Y de N elementos tales que por cada renglon j de la matriz X, se devuelva el j-ésimo valor del vector Y tal que su valor sea sign(k_0 + k_1 * x_1 + k_2 * x_2).

In [4]:
def discriminante_lineal(k_0, k_1, k_2, X):

    Y = []

    for i in xrange(X.shape[0]):
        x_1 = X[i][1]
        x_2 = X[i][2]
        Y.append(np.sign(k_0 + k_1 * x_1 + k_2 * x_2))

    return Y


print "3: discriminante lineal: \n"
Y = discriminante_lineal(k_0, k_1, k_2, X)
print Y

3: discriminante lineal: 

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


4.Desarrolla una función PLA que implemente el algoritmo de aprendizaje del perceptrón para encontrar w_0, w_1 y w_2.

In [5]:
def PLA(X, Y):

    max_iter = 0
    w = np.random.uniform(0, 1, size=X.shape[1])
    Ye = np.sign(np.dot(X, w)) 
    error = Ye * Y
    
    while any(e != 1 for e in error) and max_iter < 1000:
        i = np.random.choice(X.shape[0])
        sign = np.sign(np.dot(X[i], w))

        if sign == Y[i]:
            pass
        else:
            w = w + np.dot(X[i], Y[i])
            Ye = np.sign(np.dot(X, w)) 
            error = Ye * Y

        max_iter += 1

    return w


print "4: PLA: \n"
w_0, w_1, w_2 = PLA(X, Y)
print w_0
print w_1
print w_2

4: PLA: 

0.898656772617
0.325382715342
0.368951695134


5.Desarrolla una función error_clasificacion que reciba un vector de valores Yy un vector de valores Y_e y calcule el porcentaje de valores diferentes entre ambos vectores

In [6]:
def error_clasificacion(Y, Y_e):

    c = 0
    for i in xrange(len(Y)):
        if Y[i] != Y_e[i]:
            c += 1

    return c / float(len(Y))

print "5: error clasificacion: \n"
E_i = error_clasificacion(Y, Y)
print E_i

5: error clasificacion: 

0.0


6.Prueba que el conjunto funciona, esto es, para diferentes valores de Ny repetidos tantas veces como sea necesario al realizar lo siguiente:

k_0, k_1, k_2 = modelo_aleatorio()
X = genera_datos(N)
Y = discriminante_lineal(k_0, k_1, k_2, X)
w_0, w_1, w_2 = PLA(X, Y)
Y_e = discriminante_lineal(w_0, w_1, w_2, X)
E_i = error_clasificacion(Y, Y_e)
en todos los casos e debe de ser 0 o un valor muy cercano (como 1e-12).

In [7]:
k_0, k_1, k_2 = modelo_aleatorio()
X = genera_datos(10000)
Y = discriminante_lineal(k_0, k_1, k_2, X)
w_0, w_1, w_2 = PLA(X, Y)
Y_e = discriminante_lineal(w_0, w_1, w_2, X)
E_i = error_clasificacion(Y, Y_e)

print "6: prueba E_i: \n"
print E_i

6: prueba E_i: 

0.0173


7.E_i Es en este caso el error en muestra, que es el que se obtiene de verificar el error que el clasificador (descrito por w_0, w_1 y w_2) presenta respecto a los datos reales, pero únicamente de datos de aprendizaje.

Este error no es exactamente el error fuera de muestra, y para calcular dicho error en el plano [0, 1] x [0, 1] hay que realizar algunas operaciones de geometría analítica que no siempre son sencillas. Por esta razón vamos a considerar estimar el E_out, con un conjunto de datos sensiblemente mayor al que utilizamos para el aprendizaje.
   ```

In [8]:
X_o = genera_datos(10000)
Y_o = discriminante_lineal(k_0, k_1, k_2, X_o)
Y_eo = discriminante_lineal(w_0, w_1, w_2, X_o)
E_o = error_clasificacion(Y_o, Y_eo)

print "7: prueba E_o: \n"
print E_o

7: prueba E_o: 

0.0153


8.Ahora vamos a comparar con un número diferente de datos de aprendizaje, como E_o cambia en terminos generales.

In [21]:
def E_o_prom(N):
    
    E_lista = np.zeros(500)
    X_o = genera_datos(10000)
    
    for epoch in range(1,100):
        k_0, k_1, k_2 = modelo_aleatorio()
        Y_o = discriminante_lineal(k_0, k_1, k_2, X_o)
        
        for iter in range(1,100):
            X = genera_datos(N)
            Y = discriminante_lineal(k_0, k_1, k_2, X)
            w_0, w_1, w_2 = PLA(X, Y)
            Y_eo = discriminante_lineal(w_0, w_1, w_2, X_o)
            E_o = error_clasificacion(Y_o, Y_eo)
            
            # puse un if ya que el indice se salia de rango y marcaba error
            indice = 100 * epoch + iter
            if indice < len(E_lista):
                E_lista[indice] = E_o
            
    return sum(E_lista) / len(E_lista)

print "8: E_o_prom: \n"
print E_o_prom(5)

8: E_o_prom: 

0.0223842


9.Explica en forma gráfica que es lo que se está haciendo y que es el valor que estamos midiendo.

10.Realiza una tabla con los valores de E_o_prompara N que tome los valores de 10, 20, 50, 100, 500 respectivamente. Escribe claramente tus conclusiones de este trabajo.